pion, muon, neutrino 
https://www.feynmanlectures.caltech.edu/info/solutions/pion_muon_neutrino_sol_1.pdf

A pion (mπ = 273 me) at rest decays into a muon (mμ = 207 me) and a neutrino (mν = 0). 
Find the kinetic energy and momentum of the muon and the neutrino in MeV. 

In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi

functions for generating monomials

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

define symbolic variables

In [4]:
@polyvar p_v p_mu p_p m_mu m_p E_v E_mu E_p   

x =[p_v p_mu p_p m_mu m_p E_v E_mu E_p] 

1×8 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 p_v  p_mu  p_p  m_mu  m_p  E_v  E_mu  E_p

In [5]:
axioms= # c^2 is 1, maybe add c^2 or cs(=c^2) later on ...
[
    p_v - p_mu,                       # consevation of momentum (pion at rest)    
    E_p - m_p,                        # relativity mass energy relation (pion is at rest, energy is equal mass) 
    E_v - p_v,                        # neutrino is massless its energy equals its momentum, E p ν = ν 
    #E_mu - m_p + p_v,                # conservation of energy 
    E_p - E_mu - E_v,                 # conservation of energy
    E_mu^2 - p_mu^2 - m_mu^2          # fundamental kinematic equation for the muon 
]

5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 -p_mu + p_v
 E_p - m_p
 E_v - p_v
 E_p - E_mu - E_v
 E_mu² - m_mu² - p_mu²

In [6]:
deg = 3
deg_overall = 6

theDegrees = [2 2 2 3 3 3 3 3 ] # p_v p_mu p_p m_mu m_p E_v E_mu E_p   

candidate_mons = [
    mons_of_max_degree_and_unit_overall(x, deg, deg_overall, [])
    #all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
mons_q = mons_of_max_degree_and_unit_overall([p_v m_p  m_mu ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [p_v]).>0]) == 4.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

size.(candidate_mons) = [(2643,), (2643,), (2643,), (2643,), (2643,)]
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-04
size.(coeff_αs) = [(2643,), (2643,), (2643,), (2643,), (2643,)]
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9616 rows, 13269 columns and 31808 nonzeros
Model fingerprint: 0xb68270fc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 9474 rows and 13123 columns
Presolve time: 0.01s
Presolved: 142 rows, 146 columns, 464 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   2.400000e-05      0s

Use crossover to convert LP

OPTIMAL::TerminationStatusCode = 1

In [7]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#31 (generic function with 1 method)

In [8]:
value_q = value_poly(q)
value_q

-2.0m_p² + 2.0m_mu² + 4.0p_vm_p

In [32]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[-4.0E_p + 2.0E_v + 4.0m_p + 2.0p_mu, 2.0E_p + 2.0m_p - 4.0p_mu, -4.0E_p + 2.0E_v + 2.0p_mu, -2.0E_p - 2.0E_mu + 2.0E_v, -2.0]


5-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 -4.0E_p + 2.0E_v + 4.0m_p + 2.0p_mu
 2.0E_p + 2.0m_p - 4.0p_mu
 -4.0E_p + 2.0E_v + 2.0p_mu
 -2.0E_p - 2.0E_mu + 2.0E_v
 -2.0

the solution for p_v (neutrino momentum) above can be rewritten as  p_v  = (m_p^2 - m_u^2)/2 m_p as expected 
